In [1]:
from huggingface_hub import login
from dotenv import load_dotenv
import os
import torch
from huggingface_hub import hf_hub_download
from diffusers import FluxPipeline
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/opt/anaconda3/envs/llms/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [2]:
# !!pip install diffusers transformers accelerate

In [3]:
load_dotenv()
hf_token = os.getenv('HF_TOKEN')
login(hf_token,add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
import torch

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [5]:
classifier = pipeline('sentiment-analysis', device=device)
result = classifier('I am grateful foe everything I have')
print(result)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps


[{'label': 'POSITIVE', 'score': 0.9974709749221802}]


In [6]:
ner = pipeline('ner',device=device)
result = ner('Dr. Amir Ghadami is a famouse Businessman and Scientist')
print(result)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps


[{'entity': 'I-PER', 'score': 0.9993376, 'index': 3, 'word': 'Amir', 'start': 4, 'end': 8}, {'entity': 'I-PER', 'score': 0.99954885, 'index': 4, 'word': 'G', 'start': 9, 'end': 10}, {'entity': 'I-PER', 'score': 0.9680944, 'index': 5, 'word': '##had', 'start': 10, 'end': 13}, {'entity': 'I-PER', 'score': 0.99728763, 'index': 6, 'word': '##ami', 'start': 13, 'end': 16}]


In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct",trust_remote_code = True)

In [8]:
tokens = tokenizer.encode('I will be a multi millionaire by the age of 40, Also I will become a PhD at university of Toronto')

In [9]:
print(tokens)

[40, 686, 387, 264, 7299, 88944, 553, 279, 4231, 315, 220, 19, 15, 11, 7281, 358, 686, 3635, 264, 29561, 518, 12103, 315, 14632]


In [10]:
print(tokenizer.decode(tokens))

I will be a multi millionaire by the age of 40, Also I will become a PhD at university of Toronto


In [11]:
print(tokenizer.batch_decode(tokens))

['I', ' will', ' be', ' a', ' multi', ' millionaire', ' by', ' the', ' age', ' of', ' ', '4', '0', ',', ' Also', ' I', ' will', ' become', ' a', ' PhD', ' at', ' university', ' of', ' Toronto']


In [12]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Tell a light-hearted joke for a room of Data Scientists"}
  ]

In [13]:
tokenizer.apply_chat_template(messages,tokenizer=False)

[151644,
 8948,
 198,
 2610,
 525,
 264,
 10950,
 17847,
 151645,
 198,
 151644,
 872,
 198,
 40451,
 264,
 3100,
 69295,
 21646,
 369,
 264,
 3054,
 315,
 2885,
 56016,
 151645,
 198]

In [14]:
PHI3 = "microsoft/Phi-3-mini-4k-instruct"
GEMMA2 = "google/gemma-2-2b-it"
QWEN2 = "Qwen/Qwen2-7B-Instruct" 
MIXTRAL = "mistralai/Mixtral-8x7B-Instruct-v0.1" 
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [15]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Tell a light-hearted joke for a room of Data Scientists"}
  ]

In [16]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [17]:
tokenizer = AutoTokenizer.from_pretrained(PHI3)
tokenizer.pad_token = tokenizer.eos_token

In [18]:
inputs = tokenizer.apply_chat_template(messages,return_tensors='pt').to(device)

In [19]:
inputs

tensor([[32006,   887,   526,   263,  8444, 20255, 32007, 32010, 24948,   263,
          3578, 29899, 23057,   287,  2958,   446,   363,   263,  5716,   310,
          3630, 23753,  2879, 32007, 32000]], device='mps:0')

In [20]:
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk.


In [21]:
import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

In [22]:
AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"


In [23]:
OPENAI_API_TOKEN = os.getenv('OPENAI_API_KEY')

In [ ]:
openai = OpenAI()
def downlo

In [27]:
audio_file = open('/Users/amir/Desktop/GitHub/EchoMinutes/denver_extract.mp3','rb')

In [28]:
transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL,file=audio_file,response_format='text')

In [31]:
import requests
from bs4 import BeautifulSoup

def scrape_mp3_link(page_url):
    response = requests.get(page_url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    audio_tags = soup.find_all("audio")
    for tag in audio_tags:
        if tag.get("src"):
            return tag["src"]

    for link in soup.find_all("a"):
        if link.get("href", "").endswith(".mp3"):
            return link["href"]

    return None

# Example
page = "https://www.bbc.co.uk/sounds"
audio_url = scrape_mp3_link(page)

if audio_url:
    audio_data = requests.get(audio_url)
    with open("news_audio.mp3", "wb") as f:
        f.write(audio_data.content)
    print("✅ MP3 downloaded!")
else:
    print("❌ No MP3 found on page.")

❌ No MP3 found on page.


In [2]:
import os

def download_audio_from_url(url, output_name="news_clip.mp3"):
    os.system(f'yt-dlp -x --audio-format mp3 -o "{output_name}" "{url}"')
    print(f"✅ Downloaded: {output_name}")

# Example (NPR news)
url = "https://www.wsj.com/audio"
download_audio_from_url(url)

✅ Downloaded: news_clip.mp3


sh: yt-dlp: command not found


In [3]:
import os
import sys
import yt_dlp

def download_audio(youtube_url, output_filename="downloaded_audio.mp3"):
    # Options for audio-only download in mp3 format
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': output_filename,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'quiet': False
    }

    print(f"🎧 Downloading audio from: {youtube_url}")
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])
    print(f"✅ Done! Audio saved to: {output_filename}")

if __name__ == "__main__":
    if len(sys.argv) < 2:
        print("❌ Please provide a YouTube URL.\nUsage: python download_youtube_audio.py https://youtube.com/watch?v=...")
    else:
        youtube_url = sys.argv[1]
        download_audio(youtube_url)

ModuleNotFoundError: No module named 'yt_dlp'

In [1]:
import yt_dlp

ModuleNotFoundError: No module named 'yt_dlp'